## 基于规则或词典的信息抽取
### 1. 规则：
    对于有一定规律的实体或一定结构的文本，使用正则表达式进行模式匹配，抽取其中的实体。如：
        邮箱：[a-zA-Z0-9_-]+@[a-zA-Z0-9_-]+(?:\.[a-zA-Z0-9_-]+)
        手机号：1(3|4|5|6|7|8|9)\d{9}
        身份证号：[1-9]\d{5}(18|19|([23]\d))\d{2}((0[1-9])|(10|11|12))(([0-2][1-9])|10|20|30|31)\d{3}[0-9Xx]
    其他实体也可以总结规律编写相应的正则表达式。

In [1]:
import re

# 手机号码的正则表达式
phone_pattern = '1(3|4|5|6|7|8|9)\d{9}'
# 编译正则
phone_re = re.compile(phone_pattern)

def find_fre(text):
    fres = []
    # 找到所有匹配的结果
    for x in phone_re.finditer(text):
        beg, end = x.span()
        end = end + 1
        context = x.group()
        fres.append((beg, end, context))
    return fres

In [2]:
sentences = ['请拨打13512519088',
             '我的电话号码13966121988']

for sentence in sentences:
    fres = find_fre(sentence)
    print(sentence)
    print(fres)

请拨打13512519088
[(3, 15, '13512519088')]
我的电话号码13966121988
[(6, 18, '13966121988')]


### 2. 词典：
    对于内容比较固定、变化较少的实体，使用ac自动机进行搜索。如：   
        银行：招商银行、工商银行；     
        电影名：《海角七号》、《查理的战争》、《生化危机4》；  
        固定的英文缩写：dota、dota2、war3
    使用词典的方式，前提是有高质量的领域词典
    也可以根据已标注的实体，整理出对应的词典
    AC自动机匹配出来的有重叠实体，可以根据实体长度选取长度最长的实体或者其他策略进行选择

In [3]:
import ahocorasick

# 整理出的词表
banks = ['招商银行', '工商银行', '交通银行', '农业银行', '中国人民银行', '兴业银行', '浦发银行']
movies = ['《海角七号》', '《查理的战争》', '《生化危机4》', '《纳尼亚传奇2》', '《夺宝奇兵4》']

# 创建词典
word_dict = {}
for w in banks:
    word_dict[w] = 'company'
for w in movies:
    word_dict[w] = 'movie'
words = list(word_dict.keys())

# 构建AC自动机
AC = ahocorasick.Automaton()
for idx, key in enumerate(words):
    AC.add_word(key, (idx, key))
AC.make_automaton()

In [4]:
# 对文本使用ac进行全匹配
def ac_search(text, ac, word_dict):
    entitys = []
    for end, (_, word) in ac.iter(text):
        beg = end - len(word) + 1
        tag = word_dict[word]
        entitys.append((beg, end + 1, tag, word))
    return entitys

In [5]:
sentences = ['交通银行创始于1908年，为中国六大银行之一。',
             '《纳尼亚传奇2》是由安德鲁·亚当森执导，安德鲁·亚当森和克里斯托弗·马库斯共同担任编剧']

for sentence in sentences:
    entitys = ac_search(sentence, AC, word_dict)
    print(sentence)
    print(entitys)

交通银行创始于1908年，为中国六大银行之一。
[(0, 4, 'company', '交通银行')]
《纳尼亚传奇2》是由安德鲁·亚当森执导，安德鲁·亚当森和克里斯托弗·马库斯共同担任编剧
[(0, 8, 'movie', '《纳尼亚传奇2》')]
